In [11]:
# Fraud Detection Model Testing Script
# Test your trained model with sample transactions

import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("FRAUD DETECTION MODEL - TESTING SCRIPT")
print("="*70)

# ============================================================================
# STEP 1: LOAD THE SAVED MODELS
# ============================================================================

def load_models():
    """Load all saved model components"""
    try:
        # Load the trained model
        with open('fraud_detection_model_random_forest.pkl', 'rb') as f:
            model = pickle.load(f)
        print("✓ Model loaded successfully")
        
        # Load the scaler
        with open('scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        print("✓ Scaler loaded successfully")
        
        # Load label encoders
        with open('label_encoders.pkl', 'rb') as f:
            le_dict = pickle.load(f)
        print("✓ Label encoders loaded successfully")
        
        return model, scaler, le_dict
    
    except FileNotFoundError as e:
        print(f"\n❌ Error: {e}")
        print("Please make sure you've run the training script first!")
        return None, None, None

model, scaler, le_dict = load_models()

# ============================================================================
# STEP 2: CREATE SAMPLE TEST TRANSACTIONS
# ============================================================================

def create_sample_transactions():
    """Create sample transactions for testing (mix of normal and suspicious)"""
    
    samples = [
        # Transaction 1: NORMAL - Small withdrawal from active account
        {
            'TransactionAmount': 150.00,
            'TransactionDate': '15/09/2024 14:30',
            'TransactionType': 'Withdrawal',
            'Location': 'New York',
            'Channel': 'ATM',
            'CustomerAge': 35,
            'CustomerOccupation': 'Teacher',
            'TransactionDuration': 45,
            'LoginAttempts': 1,
            'AccountBalance': 5000.00,
            'PreviousTransactionDate': '10/09/2024 10:20',
            'Sender Country': 'USA',
            'Receiver Country': 'USA',
            'Sender Currency': 'USD',
            'Receiver Currency': 'USD',
            'Account Status': 'Active',
            'Invalid Pin Status': 'Valid',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 0
        },
        
        # Transaction 2: SUSPICIOUS - Flagged account with high retry count
        {
            'TransactionAmount': 3500.00,
            'TransactionDate': '16/09/2024 02:15',
            'TransactionType': 'Transfer',
            'Location': 'Miami',
            'Channel': 'Online',
            'CustomerAge': 28,
            'CustomerOccupation': 'Student',
            'TransactionDuration': 180,
            'LoginAttempts': 5,
            'AccountBalance': 4200.00,
            'PreviousTransactionDate': '15/09/2024 23:45',
            'Sender Country': 'USA',
            'Receiver Country': 'Nigeria',
            'Sender Currency': 'USD',
            'Receiver Currency': 'EUR',
            'Account Status': 'Flagged',
            'Invalid Pin Status': 'Locked',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 3
        },
        
        # Transaction 3: SUSPICIOUS - Suspended account, low balance
        {
            'TransactionAmount': 8000.00,
            'TransactionDate': '17/09/2024 03:30',
            'TransactionType': 'Transfer',
            'Location': 'Los Angeles',
            'Channel': 'Mobile',
            'CustomerAge': 22,
            'CustomerOccupation': 'Unemployed',
            'TransactionDuration': 200,
            'LoginAttempts': 4,
            'AccountBalance': 1500.00,
            'PreviousTransactionDate': 'NaN/NaN/NaN NaN:NaN',
            'Sender Country': 'USA',
            'Receiver Country': 'Russia',
            'Sender Currency': 'USD',
            'Receiver Currency': 'GBP',
            'Account Status': 'Suspended',
            'Invalid Pin Status': 'Invalid',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 2
        },
        
        # Transaction 4: NORMAL - Regular credit transaction
        {
            'TransactionAmount': 2500.00,
            'TransactionDate': '18/09/2024 11:00',
            'TransactionType': 'Credit',
            'Location': 'Boston',
            'Channel': 'Branch',
            'CustomerAge': 45,
            'CustomerOccupation': 'Engineer',
            'TransactionDuration': 30,
            'LoginAttempts': 1,
            'AccountBalance': 15000.00,
            'PreviousTransactionDate': '12/09/2024 14:20',
            'Sender Country': 'USA',
            'Receiver Country': 'USA',
            'Sender Currency': 'USD',
            'Receiver Currency': 'USD',
            'Account Status': 'Active',
            'Invalid Pin Status': 'Valid',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 0
        },
        
        # Transaction 5: VERY SUSPICIOUS - Night transaction, cross-border, locked PIN
        {
            'TransactionAmount': 5000.00,
            'TransactionDate': '19/09/2024 23:45',
            'TransactionType': 'Transfer',
            'Location': 'Chicago',
            'Channel': 'Online',
            'CustomerAge': 19,
            'CustomerOccupation': 'Student',
            'TransactionDuration': 250,
            'LoginAttempts': 6,
            'AccountBalance': 5500.00,
            'PreviousTransactionDate': '19/09/2024 23:30',
            'Sender Country': 'USA',
            'Receiver Country': 'China',
            'Sender Currency': 'USD',
            'Receiver Currency': 'EUR',
            'Account Status': 'Flagged',
            'Invalid Pin Status': 'Locked',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 3
        },
        
        # Transaction 6: NORMAL - Regular POS purchase
        {
            'TransactionAmount': 85.50,
            'TransactionDate': '20/09/2024 18:20',
            'TransactionType': 'Debit',
            'Location': 'Seattle',
            'Channel': 'POS',
            'CustomerAge': 38,
            'CustomerOccupation': 'Doctor',
            'TransactionDuration': 25,
            'LoginAttempts': 1,
            'AccountBalance': 12000.00,
            'PreviousTransactionDate': '18/09/2024 12:30',
            'Sender Country': 'USA',
            'Receiver Country': 'USA',
            'Sender Currency': 'USD',
            'Receiver Currency': 'USD',
            'Account Status': 'Active',
            'Invalid Pin Status': 'Valid',
            'Invalid pin retry limits': 3,
            'Invalid pin retry count': 0
        }
    ]
    
    return pd.DataFrame(samples)

# Create sample transactions
test_df = create_sample_transactions()
print(f"\n✓ Created {len(test_df)} sample transactions for testing\n")

# ============================================================================
# STEP 3: PREPROCESS TEST DATA (SAME AS TRAINING)
# ============================================================================

def preprocess_transaction(df):
    """Apply the same preprocessing as training"""
    
    df = df.copy()
    
    # Convert dates
    df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='%d/%m/%Y %H:%M')
    df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'], 
                                                    format='%d/%m/%Y %H:%M', 
                                                    errors='coerce')
    
    # Extract time features
    df['Hour'] = df['TransactionDate'].dt.hour
    df['DayOfWeek'] = df['TransactionDate'].dt.dayofweek
    df['Month'] = df['TransactionDate'].dt.month
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
    df['IsNightTime'] = ((df['Hour'] >= 22) | (df['Hour'] <= 6)).astype(int)
    
    # Time since previous transaction
    df['HoursSincePrevTransaction'] = (
        (df['TransactionDate'] - df['PreviousTransactionDate']).dt.total_seconds() / 3600
    )
    df['HoursSincePrevTransaction'].fillna(df['HoursSincePrevTransaction'].median(), inplace=True)
    
    # Additional features
    df['AmountToBalanceRatio'] = df['TransactionAmount'] / (df['AccountBalance'] + 1)
    df['IsCrossBorder'] = (df['Sender Country'] != df['Receiver Country']).astype(int)
    df['IsCurrencyMismatch'] = (df['Sender Currency'] != df['Receiver Currency']).astype(int)
    
    # Encode categorical variables
    categorical_cols = ['TransactionType', 'Location', 'Channel', 
                        'CustomerOccupation', 'Sender Country', 'Receiver Country',
                        'Sender Currency', 'Receiver Currency', 'Account Status',
                        'Invalid Pin Status']
    
    for col in categorical_cols:
        if col in le_dict:
            # Handle unknown categories
            df[col] = df[col].apply(lambda x: x if x in le_dict[col].classes_ else le_dict[col].classes_[0])
            df[col] = le_dict[col].transform(df[col])
    
    # Drop non-feature columns
    features_to_drop = ['TransactionDate', 'PreviousTransactionDate']
    df = df.drop(features_to_drop, axis=1, errors='ignore')
    
    return df

# Preprocess test data
test_processed = preprocess_transaction(test_df)
print("✓ Test data preprocessed successfully\n")

# ============================================================================
# STEP 4: MAKE PREDICTIONS
# ============================================================================

def predict_fraud_batch(transactions_df):
    """Predict fraud for multiple transactions"""
    
    # Scale features
    X_test = scaler.transform(transactions_df)
    
    # Make predictions
    predictions = model.predict(X_test)
    probabilities = model.predict_proba(X_test)[:, 1]
    
    return predictions, probabilities

# Get predictions
predictions, probabilities = predict_fraud_batch(test_processed)

# ============================================================================
# STEP 5: DISPLAY RESULTS
# ============================================================================

print("="*70)
print("PREDICTION RESULTS")
print("="*70)

results_df = test_df[['TransactionAmount', 'TransactionType', 'Account Status', 
                      'Invalid Pin Status', 'LoginAttempts']].copy()
results_df['Fraud_Prediction'] = predictions
results_df['Fraud_Probability'] = (probabilities * 100).round(2)
results_df['Risk_Level'] = pd.cut(probabilities, 
                                   bins=[0, 0.3, 0.6, 1.0],
                                   labels=['Low', 'Medium', 'High'])

# Display each transaction
for idx, row in results_df.iterrows():
    print(f"\n{'─'*70}")
    print(f"TRANSACTION #{idx + 1}")
    print(f"{'─'*70}")
    print(f"Amount:              ${row['TransactionAmount']:,.2f}")
    print(f"Type:                {test_df.loc[idx, 'TransactionType']}")
    print(f"Account Status:      {test_df.loc[idx, 'Account Status']}")
    print(f"PIN Status:          {test_df.loc[idx, 'Invalid Pin Status']}")
    print(f"Login Attempts:      {row['LoginAttempts']}")
    print(f"\n{'─'*30}")
    
    if row['Fraud_Prediction'] == 1:
        print(f"⚠️  PREDICTION:        FRAUD DETECTED")
        print(f"🔴 Risk Level:        {row['Risk_Level']}")
        print(f"📊 Fraud Probability: {row['Fraud_Probability']:.2f}%")
        print(f"{'─'*30}")
        print("🚨 ACTION REQUIRED: Review this transaction immediately!")
    else:
        print(f"✅ PREDICTION:        LEGITIMATE")
        print(f"🟢 Risk Level:        {row['Risk_Level']}")
        print(f"📊 Fraud Probability: {row['Fraud_Probability']:.2f}%")
        print(f"{'─'*30}")
        print("✓ Transaction appears safe to proceed.")

# Summary statistics
print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
print(f"Total Transactions:   {len(results_df)}")
print(f"Flagged as Fraud:     {predictions.sum()}")
print(f"Marked as Legitimate: {len(predictions) - predictions.sum()}")
print(f"Average Fraud Risk:   {probabilities.mean() * 100:.2f}%")

# ============================================================================
# STEP 6: FUNCTION TO TEST SINGLE TRANSACTION
# ============================================================================

def test_single_transaction(transaction_dict):
    """
    Test a single transaction
    
    Parameters:
    transaction_dict: Dictionary containing transaction details
    
    Returns:
    prediction, probability, and risk assessment
    """
    # Create DataFrame from single transaction
    single_df = pd.DataFrame([transaction_dict])
    
    # Preprocess
    processed = preprocess_transaction(single_df)
    
    # Scale
    scaled = scaler.transform(processed)
    
    # Predict
    prediction = model.predict(scaled)[0]
    probability = model.predict_proba(scaled)[0][1]
    
    # Risk level
    if probability < 0.3:
        risk_level = "Low"
    elif probability < 0.6:
        risk_level = "Medium"
    else:
        risk_level = "High"
    
    return {
        'is_fraud': bool(prediction),
        'probability': float(probability),
        'risk_level': risk_level,
        'recommendation': 'Block Transaction' if prediction else 'Allow Transaction'
    }

# ============================================================================
# STEP 7: EXAMPLE - TEST A NEW CUSTOM TRANSACTION
# ============================================================================

print(f"\n{'='*70}")
print("TESTING CUSTOM TRANSACTION")
print(f"{'='*70}")

# Create a custom suspicious transaction
custom_transaction = {
    'TransactionAmount': 9500.00,
    'TransactionDate': '21/09/2024 01:30',
    'TransactionType': 'Transfer',
    'Location': 'Denver',
    'Channel': 'Online',
    'CustomerAge': 25,
    'CustomerOccupation': 'Student',
    'TransactionDuration': 300,
    'LoginAttempts': 7,
    'AccountBalance': 2000.00,
    'PreviousTransactionDate': '21/09/2024 01:15',
    'Sender Country': 'USA',
    'Receiver Country': 'Unknown',
    'Sender Currency': 'USD',
    'Receiver Currency': 'BTC',
    'Account Status': 'Flagged',
    'Invalid Pin Status': 'Locked',
    'Invalid pin retry limits': 3,
    'Invalid pin retry count': 3
}

result = test_single_transaction(custom_transaction)

print("\nCustom Transaction Details:")
print(f"Amount: ${custom_transaction['TransactionAmount']:,.2f}")
print(f"Time: {custom_transaction['TransactionDate']}")
print(f"Type: {custom_transaction['TransactionType']}")
print(f"\nPrediction Results:")
print(f"Is Fraud: {result['is_fraud']}")
print(f"Probability: {result['probability']*100:.2f}%")
print(f"Risk Level: {result['risk_level']}")
print(f"Recommendation: {result['recommendation']}")

print(f"\n{'='*70}")
print("TESTING COMPLETE!")
print(f"{'='*70}")
print("\nYou can now use test_single_transaction() function to test")
print("any new transaction by passing a dictionary with transaction details!")

FRAUD DETECTION MODEL - TESTING SCRIPT

❌ Error: [Errno 2] No such file or directory: 'fraud_detection_model_random_forest.pkl'
Please make sure you've run the training script first!

✓ Created 6 sample transactions for testing



TypeError: argument of type 'NoneType' is not iterable